In [2]:
import pandas as pd

credits = pd.read_parquet('data/credits.parquet')
projects = pd.read_parquet('data/projects.parquet')

issuances = credits[credits['transaction_type'] == 'issuance'].copy()
issuances.loc[:, 'issuance_yr'] = issuances.transaction_date.dt.year

In [3]:
ldcs = ["Angola",
        "Benin",
        "Burkina Faso",
        "Burundi",
        "Central African Republic",
        "Chad",
        "Comoros",
        "DR Congo",
        "Djibouti",
        "Eritrea",
        "Ethiopia",
        "Gambia",
        "Guinea",
        "Guinea-Bissau",
        "Lesotho",
        "Liberia",
        "Madagascar",
        "Malawi",
        "Mali",
        "Mauritania",
        "Mozambique",
        "Niger",
        "Rwanda",
        "Senegal",
        "Sierra Leone",
        "Somalia",
        "South Sudan",
        "Sudan",
        "Togo",
        "Uganda",
        "Tanzania",
        "Zambia"
        "Afghanistan",
        "Bangladesh",
        "Cambodia",
        "Laos",
        "Myanmar",
        "Nepal",
        "Timor-Leste",
        "Yemen",
        "Haiti",
        "Kiribati",
        "Solomon Islands",
        "Tuvalu"]

In [4]:
hydro_projects = projects[
    (projects['project_type'] == 'Hydropower') & (projects['registry'] == 'verra') & ~(projects['country'].isin(ldcs))]

hydro_ids = hydro_projects.project_id.to_list()
hydro_issuances = issuances[issuances['project_id'].isin(hydro_ids)].copy()


## Figure 1 

In [5]:
ts = hydro_issuances.groupby('issuance_yr').quantity.sum()
{int(k): v for k, v in ts.to_dict().items()}
# copy / paste to `issuances.js`

{2009: 2605227,
 2010: 4991786,
 2011: 7670460,
 2012: 11857529,
 2013: 8307486,
 2014: 5508923,
 2015: 5264806,
 2016: 3310261,
 2017: 3071090,
 2018: 5971540,
 2019: 9604842,
 2020: 19355331,
 2021: 43046942,
 2022: 33783221,
 2023: 37728043,
 2024: 15870432,
 2025: 6077005}

In [21]:
project_stats = {}

project_stats['total_hydro'] = len(hydro_projects)
project_stats['issued_hydro'] = hydro_issuances.project_id.nunique()
project_stats['issued_pre_ban'] = sum(hydro_issuances.groupby('project_id').issuance_yr.min() <= 2019)
project_stats['issued_post_ban'] = sum(hydro_issuances.groupby('project_id').issuance_yr.min() > 2019)

In [22]:
post_ban_issuance = ts.loc[2020:].sum() / 1_000_000
post_ban_issuance / (ts.sum() / 1_000_000)

np.float64(0.695730507200172)

In [23]:
post_ban_total_issuance = issuances[issuances['issuance_yr'] > 2019].quantity.sum() / 1_000_000
post_ban_issuance / post_ban_total_issuance

np.float64(0.09411515944932215)